Plot the three longest whiskers from the first frame of the video.

In [ ]:
import pyarrow.parquet as pq
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import json

In [ ]:
# Data directory
data_dir = '/home/wanglab/data/whisker_asym/wa001/test'
# Load the Parquet file
parquet_file = 'WA001_080224_01_left.parquet'
table = pq.read_table(f'{data_dir}/{parquet_file}')

# Convert the table to a Pandas DataFrame
df = table.to_pandas()
df

In [ ]:
# Filter the DataFrame to include only the first frame (fid = 0)
first_frame_df = df[df['fid'] == 0]

# Sort by the "length" column and take the top 3 longest whiskers
# longest_whiskers = first_frame_df.nlargest(3, 'length')

# Sort by the "pixel_length" column and take the top 3 longest whiskers
longest_whiskers = first_frame_df.nlargest(3, 'pixel_length')
longest_whiskers

In [ ]:
# Plot each whisker
plt.figure(figsize=(8, 6))

for index, row in longest_whiskers.iterrows():
    # Pixels for the whisker
    pixels_x = row['pixels_x']
    pixels_y = row['pixels_y']
    
    # Follicle coordinates
    follicle_x = row['follicle_x']
    follicle_y = row['follicle_y']
    
    # Adjust coordinates to use follicle as origin
    adjusted_pixels_x = [x - follicle_x for x in pixels_x]
    adjusted_pixels_y = [y - follicle_y for y in pixels_y]
    
    # Plot the whisker
    # plt.plot(adjusted_pixels_x, adjusted_pixels_y, label=f'Whisker {row["wid"]} (Length: {row["length"]})')
    plt.plot(pixels_x, pixels_y, label=f'Whisker {row["wid"]} (Length: {row["length"]})')
    
# Set plot labels and title
plt.xlabel('X pixels (adjusted)')
plt.ylabel('Y pixels (adjusted)')
plt.title('Top 3 Longest Whiskers from First Frame (fid = 0)')
plt.legend()
plt.grid(True)
# Reverse the y-axis
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# Read in a frame of the video
video_name = 'WA001_080224_01_TopCam_10s.mp4'
frame_num = 0
cap = cv2.VideoCapture(f'{data_dir}/{video_name}')
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
ret, frame = cap.read()
# Display the frame
plt.figure(figsize=(8, 6))
plt.imshow(frame)

In [ ]:
# Create set of colors for up to 20 whiskers, starting with red, green, blue
colors = [(255,0,0), (0,255,0), (0,0,255), (255,255,0), (255,0,255),
            (0,255,255), (128,0,0), (0,128,0), (0,0,128), (128,128,0),
            (128,0,128), (0,128,128), (64,0,0), (0,64,0), (0,0,64),
            (64,64,0), (64,0,64), (0,64,64), (192,0,0), (0,192,0)]

In [ ]:
face_side = 'left'
base_name = 'WA001_080224_01'
# If using split data (left or right whiskers), open the corresponding whiskerpad_{side}.json file
with open(f'{data_dir}/whiskerpad_{os.path.basename(base_name).split(".")[0]}.json', 'r') as f:
    whiskerpad_params = json.load(f)

# Enumerate whiskerpads field and find the "Location" value for the whiskerpad that has "FaceSide" value of face_side
for i, whiskerpad in enumerate(whiskerpad_params['whiskerpads']):
    if whiskerpad['FaceSide'].lower() == face_side:
        whiskerpad_location = whiskerpad['Location']
        image_coord = whiskerpad['ImageCoordinates']
        break

# Convert whiskerpad_location to a NumPy array
whiskerpad_location = np.array(whiskerpad_location)

whiskerpad_location

In [ ]:
# Using index of three longest whiskers, plot the whiskers on the frame
for index, whisker_data in longest_whiskers.iterrows():
    color_index = index % len(colors)
    color = colors[color_index]
    print(f"Whisker ID: {whisker_data['wid']}, color: {color}")

    print(whisker_data['pixels_x'][0], whisker_data['pixels_y'][0])
    for j in range(whisker_data['pixels_x'].shape[0]):
        # add the image_coord first two values
        x = int(whisker_data['pixels_x'][j]) + image_coord[0]
        y = int(whisker_data['pixels_y'][j]) + image_coord[1]
        cv2.circle(frame, (x, y), 2, color, -1)

In [ ]:
# Same thing for right side
face_side = 'right'
base_name = 'WA001_080224_01'
# If using split data (left or right whiskers), open the corresponding whiskerpad_{side}.json file
with open(f'{data_dir}/whiskerpad_{os.path.basename(base_name).split(".")[0]}.json', 'r') as f:
    whiskerpad_params = json.load(f)

# Enumerate whiskerpads field and find the "Location" value for the whiskerpad that has "FaceSide" value of face_side
for i, whiskerpad in enumerate(whiskerpad_params['whiskerpads']):
    if whiskerpad['FaceSide'].lower() == face_side:
        whiskerpad_location = whiskerpad['Location']
        image_coord = whiskerpad['ImageCoordinates']
        break

# Convert whiskerpad_location to a NumPy array
whiskerpad_location = np.array(whiskerpad_location)

# Data directory
data_dir = '/home/wanglab/data/whisker_asym/wa001/test'
# Load the Parquet file
parquet_file = 'WA001_080224_01_right.parquet'
table = pq.read_table(f'{data_dir}/{parquet_file}')

# Convert the table to a Pandas DataFrame
df = table.to_pandas()
# Filter the DataFrame to include only the first frame (fid = 0)
first_frame_df = df[df['fid'] == 0]

# Sort by the "pixel_length" column and take the top 3 longest whiskers
longest_whiskers = first_frame_df.nlargest(3, 'pixel_length')

# Using index of three longest whiskers, plot the whiskers on the frame
for index, whisker_data in longest_whiskers.iterrows():
    color_index = index % len(colors)
    color = colors[color_index]
    print(f"Whisker ID: {whisker_data['wid']}, color: {color}")

    print(whisker_data['pixels_x'][0], whisker_data['pixels_y'][0])
    for j in range(whisker_data['pixels_x'].shape[0]):
        # add the image_coord first two values
        x = int(whisker_data['pixels_x'][j]) + image_coord[0]
        y = int(whisker_data['pixels_y'][j]) + image_coord[1]
        cv2.circle(frame, (x, y), 2, color, -1)

In [ ]:
# Plot the frame below the cell in the notebook
plt.imshow(frame)